In [1]:
import tifffile
import numpy as np
import pandas as pd
import cv2
from scipy.ndimage import center_of_mass, extrema
import napari 
import matplotlib.pyplot as plt
from skimage import measure
import os

In [2]:
scalling_x = 10 #22
scalling_y =  10#20
translation_x = 200 # 422.5
translation_y = 200 # 336
main_dir = "D:/Internship/NTU/simulation/with_center/simulation3/"

# load function and image

In [3]:
def get_coor (path_to_log_file,rescaleX,rescaleY):
    with open(path_to_log_file, "r", encoding='utf-8-sig') as f:
        string = f.read()
        lines = string.split("\n")
        num = 1
        event_num = []
        x_coordinates = []
        y_coordinates = []
        for line in lines:

            if len(line ) != 0:
                if (int(num) - 1)% 3  == 0: 
                    event_num.append(num)
                    num+=1
                
                else: 
                    y = line[1:-1].split(',')[1]
                    x = line[1:-1].split(',')[0]
                    y_coordinates.append((rescaleY * (-float(y)))+translation_x-scalling_y)
                    x_coordinates.append((rescaleX * float(x))+ translation_y-scalling_x)
                    #note: with center simulation Y -20 X -35
                    # 
                    num+=1
    return len(event_num) , pd.DataFrame({'y_coordinates': y_coordinates, 'x_coordinates': x_coordinates}) 

In [4]:
# Load tiff file and log position
#im_path = "D:/Internship/NTU/simulation/with_center/simulation1/multi_center_resized.ome.tif"
im_path = main_dir + "particle.ome.tif"
im = tifffile.imread(im_path)
rescaling_y = im.shape[1]/400
rescaling_x = im.shape[2]/400
rescaling_y,rescaling_x

(1.0, 1.0)

In [5]:
im.shape

(501, 400, 400)

In [6]:
event_all_log = []
path = main_dir + "nonzero_frames_final.log"
with open(path, "r", encoding='utf-8-sig') as f:
    string = f.read()
    lines = string.split("\n")
    for line in lines:
        if len(line ) != 0:
            events = line.split(":")[1].split(",")
            frame = line.split(":")[0].split()[1]
           
            if len(events) == 1 :
                continue
            else:
                num_events = [x.split("=")[1] for x in events]
                x= int(num_events[0]) + float(num_events[1])
                num = [float(frame),x,float(num_events[2])]
                event_all_log.append(num)

In [7]:
# frame , fusion, fission 
event_all_log = (np.transpose(np.array(event_all_log)))
print(f"all_fusion {np.sum(event_all_log[1])}")
print(f"all_fission {np.sum(event_all_log[2])}")


all_fusion 29.0
all_fission 29.0


In [8]:
event_all_log

array([[ 14.,  21.,  24.,  26.,  28.,  29.,  50.,  63.,  74.,  82.,  85.,
         86.,  87.,  88.,  89.,  93., 106., 108., 174., 179., 180., 181.,
        199., 203., 208., 214., 242., 245., 248., 272., 276., 278., 298.,
        312., 314., 317., 324., 331., 335., 336., 346., 350., 353., 354.,
        381., 382., 389., 394., 401., 405., 427., 429., 434., 455., 489.,
        499.],
       [  1.,   1.,   0.,   0.,   0.,   1.,   1.,   0.,   0.,   0.,   1.,
          1.,   0.,   1.,   1.,   1.,   0.,   1.,   0.,   0.,   0.,   1.,
          1.,   0.,   1.,   1.,   0.,   1.,   0.,   0.,   1.,   1.,   0.,
          1.,   0.,   1.,   0.,   0.,   1.,   1.,   1.,   0.,   1.,   0.,
          0.,   1.,   0.,   0.,   1.,   1.,   0.,   1.,   1.,   1.,   0.,
          0.],
       [  0.,   0.,   1.,   1.,   1.,   0.,   0.,   1.,   1.,   1.,   0.,
          0.,   1.,   0.,   0.,   0.,   1.,   0.,   1.,   1.,   1.,   0.,
          0.,   1.,   0.,   0.,   1.,   0.,   1.,   1.,   0.,   1.,   1.,
        

In [9]:
path1_fis = main_dir + "fission_position_2.log"
path1_fus = main_dir+ "/fusion_position_2.log"



num_fission,fission_coor = get_coor(path1_fis, rescaling_x, rescaling_y)
num_fusion,fusion_coor = get_coor(path1_fus, rescaling_x, rescaling_y)

print(f"num_fusion {num_fusion}")
print(f"num_fission {num_fission}")
fusion_coor


num_fusion 29
num_fission 29


,y_coordinates,x_coordinates
0,185.17895,198.402350
1,186.79624,203.525700
2,187.44127,205.299600
3,188.93201,207.224000
4,200.48440,184.595080
5,201.22530,182.674970
6,206.97460,225.398700
7,210.09840,228.249400
8,182.05516,198.327740
9,183.46390,202.917900


In [10]:
# append frame in the coordinate array. 
# each frame consistingg of at least 2 coordinates, one for each fusion and fission event
fusion_frames = []
fission_frames = []

fusion_idx = 0
fission_idx = 0

for x in range(len(event_all_log[0])):
    if event_all_log[1][x] > 0 :
        frame = event_all_log[0][x]
        r_frame = np.repeat(frame,event_all_log[1][x]*2 )
        if x == 0:
            fusion_frames = r_frame
        else:
            fusion_frames = np.concatenate((fusion_frames,r_frame))
        fusion_idx += 1
    if event_all_log[2][x] > 0:
        frame = event_all_log[0][x]
        r_frame = np.repeat(frame,event_all_log[2][x]*2 )

        if x == 0:
            fission_frames = r_frame
        else:
            fission_frames = np.concatenate((fission_frames,r_frame))

        fission_idx += 1

# Assign the lists as new columns in your DataFrames
fusion_coor['frame'] = fusion_frames
fission_coor['frame'] = fission_frames

In [11]:
# need 2 loop coz frame are different
fusion_coords = []
for f in range(len(fusion_coor['frame'])):
    frame = int(fusion_coor.iloc[f,2])
    target_frame = im[frame]
    coor = np.append( fusion_coor.iloc[f,0], fusion_coor.iloc[f,1]) # x,y coordinates or yx?
    fusion_coords.append(np.append(frame, coor))

fission_coords = []
for f in range(len(fission_coor['frame'])):
    frame = int(fission_coor.iloc[f,2])
    target_frame = im[frame]
    coor = np.append( fission_coor.iloc[f,0], fission_coor.iloc[f,1]) # x,y coordinates or yx?
    fission_coords.append(np.append(frame, coor))


In [12]:
fusion_coor

,y_coordinates,x_coordinates,frame
0,185.17895,198.402350,14.0
1,186.79624,203.525700,14.0
2,187.44127,205.299600,21.0
3,188.93201,207.224000,21.0
4,200.48440,184.595080,29.0
5,201.22530,182.674970,29.0
6,206.97460,225.398700,50.0
7,210.09840,228.249400,50.0
8,182.05516,198.327740,85.0
9,183.46390,202.917900,85.0


In [13]:
fusion_coords

[array([ 14.     , 185.17895, 198.40235]),
 array([ 14.     , 186.79624, 203.5257 ]),
 array([ 21.     , 187.44127, 205.2996 ]),
 array([ 21.     , 188.93201, 207.224  ]),
 array([ 29.     , 200.4844 , 184.59508]),
 array([ 29.     , 201.2253 , 182.67497]),
 array([ 50.    , 206.9746, 225.3987]),
 array([ 50.    , 210.0984, 228.2494]),
 array([ 85.     , 182.05516, 198.32774]),
 array([ 85.    , 183.4639, 202.9179]),
 array([ 86.    , 247.0679, 243.5438]),
 array([ 86.    , 246.83  , 247.7367]),
 array([ 88.     , 187.73803, 168.6903 ]),
 array([ 88.     , 184.45341, 171.1241 ]),
 array([ 89.    , 131.6955, 227.9121]),
 array([ 89.    , 136.5373, 229.6379]),
 array([ 93.    , 142.0061, 245.4214]),
 array([ 93.    , 137.6202, 241.7999]),
 array([108.    , 142.0061, 245.4214]),
 array([108.    , 140.5979, 247.3688]),
 array([181.    , 166.1847, 200.8907]),
 array([181.     , 169.0949 , 198.19283]),
 array([199.     , 166.5903 , 195.33877]),
 array([199.      , 166.2819  , 190.440614]),
 

In [14]:
fission_coor

,y_coordinates,x_coordinates,frame
0,135.58230,227.923200,24.0
1,136.13660,232.129100,24.0
2,274.48550,182.123730,26.0
3,278.44870,180.187350,26.0
4,207.11150,225.736400,28.0
5,209.93770,228.391900,28.0
6,139.93770,250.157300,63.0
7,142.36860,246.354000,63.0
8,184.50648,171.313400,74.0
9,187.75841,168.628000,74.0


# Load fission fusion frome my script

In [46]:
path = "D:/Internship/NTU/simulation/with_center/algo_output/center_resize_simulation_event.csv"
df = pd.read_csv(path)
df

,Volume_pre,Volume_post,Distance,Nearest_Label,y_corr,x_corr,Volume_diff,isFusion,Frame,Label
0,0.1800,0.315,27.307814,6.0,188.928571,194.000000,0.3150,1.0,0.0,1.0
1,0.1350,0.315,27.307814,6.0,188.928571,194.000000,0.3150,1.0,0.0,1.0
2,3.3750,3.555,27.307814,4.0,166.259494,173.664557,0.3150,1.0,0.0,1.0
3,8.0550,11.115,27.307814,5.0,249.475709,242.327935,0.3150,1.0,0.0,1.0
4,0.4050,11.115,27.307814,5.0,249.475709,242.327935,0.3150,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
488,7.4250,6.885,9.147006,2.0,137.490909,179.124242,-0.5400,0.0,87.0,7.0
489,0.0000,0.000,0.000000,0.0,176.000000,287.159483,-8.1450,0.0,88.0,7.0
490,0.0000,0.000,0.000000,0.0,171.771739,300.543478,-7.2000,0.0,96.0,7.0
491,1.1025,5.580,6.437347,5.0,261.584677,263.024194,0.8775,1.0,97.0,7.0


In [47]:
mask_fusion = df['isFusion'].astype(int) == 1
mask_fusion

0       True
1       True
2       True
3       True
4       True
       ...  
488    False
489    False
490    False
491     True
492     True
Name: isFusion, Length: 493, dtype: bool

In [49]:
fission_event = df[~mask_fusion]
fusion_event = df[mask_fusion]

In [50]:
df_fission = pd.DataFrame({
    'frame': fission_event['Frame'],
    'isFusion': fission_event['isFusion'],
    'x_coordinates': fission_event['x_corr'],
    'y_coordinates': fission_event['y_corr'],
})

df_fusion = pd.DataFrame({
    'frame': fusion_event['Frame'],
    'isFusion': fusion_event['isFusion'],
    'x_coordinates': fusion_event['x_corr'],
    'y_coordinates': fusion_event['y_corr'],
})

# run visualization

In [32]:
middle = np.array([1, translation_y- scalling_y, translation_x-scalling_x])[np.newaxis, :]  # shape (1, 3)
for i in range(1, im.shape[0]):
    new_row = np.array([i+1, translation_y-scalling_y, translation_x-scalling_x])[np.newaxis, :]
    middle = np.vstack((middle, new_row))

In [33]:
'''
middle = np.array([1, 200, 200])[np.newaxis, :]  # shape (1, 3)
for i in range(1, im.shape[0]):
    new_row = np.array([i+1, 200, 200])[np.newaxis, :]
    middle = np.vstack((middle, new_row))
'''


'\nmiddle = np.array([1, 200, 200])[np.newaxis, :]  # shape (1, 3)\nfor i in range(1, im.shape[0]):\n    new_row = np.array([i+1, 200, 200])[np.newaxis, :]\n    middle = np.vstack((middle, new_row))\n'

In [34]:
# Y is in the first column and X is in the second. 
test_coord = [np.array([2,0,0]), np.array([2,422.5,0]), np.array([2,845,0]), 
              np.array([2,0,336]), np.array([2,0,672]), np.array([2,422.5,336]),np.array([  2    , 302.5515, 404.3512])]
label = [0,1,2,3,4,5,9] 

In [35]:
#show image
viewer = napari.Viewer()
viewer.add_labels(im, name ="labeled image")
viewer.add_points(middle, face_color='white', size =2, name = "central point")
viewer.add_points(fusion_coords, face_color='yellow', size =2, name = "fussion")
viewer.add_points(fission_coords, face_color='green', size =2, name = "fission")
viewer.add_points(test_coord, face_color='yellow', size =1 , text = label, name = "label number")
napari.run()

#TODO is it the correct position of this iamghe

In [36]:
'''
#show image
viewer = napari.Viewer()
viewer.add_labels(im, name ="labeled image")
viewer.add_points(middle, face_color='white', size =5, name = "label number")
viewer.add_points(test_coord, face_color='yellow', size =1 , text = label, name = "label number")
napari.run()
'''


'\n#show image\nviewer = napari.Viewer()\nviewer.add_labels(im, name ="labeled image")\nviewer.add_points(middle, face_color=\'white\', size =5, name = "label number")\nviewer.add_points(test_coord, face_color=\'yellow\', size =1 , text = label, name = "label number")\nnapari.run()\n'